In [0]:
import json
from pprint import pprint
from google.colab import auth
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
import pickle

In [0]:
auth.authenticate_user()
drive_service = build('drive', 'v3')

In [0]:
def _create_file_request(file_id):
    return drive_service.files().get_media(fileId=file_id)

def _download_response_bytes(request, print_progress=False):
    downloaded = io.BytesIO()
    downloader = MediaIoBaseDownload(downloaded, request)
   
    for status in _progbar(downloader):
        if print_progress:
            print("Downloaded {}/{} bytes".format(status.resumable_progress, status.total_size))
 
    downloaded.seek(0)
    return downloaded.read()
 
 
def _progbar(downloader):
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        yield status
 
 
def get_file_id(name):
    return get_matching_files(name)[0]['id']
 
 
def move_from_drive_to_disk(file_names, file_destinations):
    for file_name, dest in zip(file_names, file_destinations):
        file_id = get_file_id(file_name)
        print('Downloading file: "{}"'.format(file_name))
        file_bytes = _download_response_bytes(_create_file_request(file_id), print_progress=True)
        with open(dest, "wb") as f:
            f.write(file_bytes)
 
 
def load_pickled_files(file_names):
    for name in file_names:
        yield pickle.load(open(name, "rb"))
       
 
def get_matching_files(name):
    drive_files_response = _download_response_bytes(drive_service.files().list())
    drive_files_response_dict = json.loads(
        drive_files_response.decode('utf-8')
    )
    drive_files_dict = drive_files_response_dict['files']
 
    matching_records = [
        record
        for record in drive_files_dict
        if record['name'] == name
    ]
 
    no_records = len(matching_records)
    if no_records == 0:
        raise ValueError('no such file: "{}" on your Google Drive'.format(name))
    elif no_records > 1:
        print('warning: multiple matches for file "{}"'.format(name))
    return matching_records

In [6]:
example_file_name = 'all_x.npy'
example_file_id = get_file_id(example_file_name)
 
move_from_drive_to_disk([example_file_name], [example_file_name])

Downloaded 73774208/73774208 bytes


In [7]:
example_file_name = 'all_y.npy'
example_file_id = get_file_id(example_file_name)
 
move_from_drive_to_disk([example_file_name], [example_file_name])

Downloaded 5763728/5763728 bytes


In [8]:
import os
os.listdir()

['.config', 'all_y.npy', 'all_x.npy', 'adc.json', 'sample_data']

In [0]:
import numpy as np
import keras
from keras.layers import Dense, Conv2D, MaxPool2D,Flatten
from keras.models import Sequential
from sklearn.model_selection import train_test_split
#from keras import optimizers
from keras.optimizers import SGD

In [19]:
x = np.load('all_x.npy')
y = np.load('all_y.npy')
print(x.shape, 'x')
print(y.shape, 'y')

(72045, 32, 32) x
(72045, 10) y


In [20]:
x = np.expand_dims(x, axis = 3)
x.shape

(72045, 32, 32, 1)

In [21]:
model = Sequential()
model.add(Conv2D(6, 5, 5, input_shape= (32,32,1)))
model.add(MaxPool2D(2, 2))
model.add(Conv2D(16, 5, 5))
model.add(MaxPool2D(2, 2))
model.add(Flatten())

model.add(Dense(120, activation = 'relu'))
model.add(Dense(84, activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), input_shape=(32, 32, 1...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5))`
  after removing the cwd from sys.path.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_5 (Dense)              (None, 84)                10164     
__________

In [0]:
sgd = SGD(lr = 0.0001, decay = 1e-6, momentum = 0.9, nesterov= True)

In [0]:
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

In [26]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 100)

Train on 57636 samples, validate on 14409 samples
Epoch 1/100
57636/57636 [==============================] - 13s 229us/step - loss: 13.3905 - acc: 0.1645 - val_loss: 12.8017 - val_acc: 0.2020
Epoch 2/100
57636/57636 [==============================] - 12s 203us/step - loss: 12.7110 - acc: 0.2068 - val_loss: 12.5912 - val_acc: 0.2147
Epoch 3/100
57636/57636 [==============================] - 12s 210us/step - loss: 12.5574 - acc: 0.2159 - val_loss: 12.4843 - val_acc: 0.2160
Epoch 4/100
57636/57636 [==============================] - 12s 212us/step - loss: 8.3802 - acc: 0.2799 - val_loss: 1.7084 - val_acc: 0.4322
Epoch 5/100
57636/57636 [==============================] - 11s 199us/step - loss: 1.3893 - acc: 0.5412 - val_loss: 1.2727 - val_acc: 0.5841
Epoch 6/100
57636/57636 [==============================] - 13s 220us/step - loss: 1.0123 - acc: 0.6714 - val_loss: 0.9842 - val_acc: 0.6817
Epoch 7/100
31840/57636 [===============>..............] - ETA: 5s - loss: 0.8358 - acc: 0.7274

KeyboardInterrupt: ignored